In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.vec_env import DummyVecEnv

from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])


import warnings
warnings.filterwarnings('ignore')
import itertools

In [2]:
# ETFs
TICKERS = ['XLP', 'XLY', 'XLI', 'XLE', 'XLK', 'IYZ', 'XRT', 'XLV', 'XLU', 'VTI']

# Mutual Funds
# TICKERS = ['VCSAX', 'FSCPX', 'VINAX', 'FSENX', 'VITAX', 'FSDCX', 'FSRPX', 'VGHCX', 'VUIAX', 'VEXAX']

# Futures
# TICKERS = ['SPSU', 'SPSD', 'SPSI', 'SPEN', 'SPTL', 'SPTS', 'SPSD', 'SPHC', 'SPUT', 'ES']

START_DATE = '1980-01-01'
END_DATE = '2024-12-31'
TRAIN_START_DATE = '2006-01-01'
TRAIN_END_DATE = '2018-12-31'
TRADE_START_DATE = '2019-01-01'
TRADE_END_DATE = END_DATE 

print('Downloading data...')
stock_data = YahooDownloader(
    ticker_list=TICKERS,
    start_date=START_DATE,
    end_date=END_DATE,
).fetch_data()

stock_data.head()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (62600, 8)


Price,date,close,high,low,open,volume,tic,day
0,1998-12-22,11.746052,11.809160,11.706609,11.769717,15200,XLE,1
1,1998-12-22,14.582214,14.582214,14.533281,14.533281,600,XLI,1
2,1998-12-22,23.943205,24.281748,23.744748,24.211705,300500,XLK,1
3,1998-12-22,14.274880,14.291714,13.938209,13.938209,150300,XLP,1
4,1998-12-22,11.913811,12.082314,11.913811,12.082314,7900,XLU,1


In [81]:
stock_data["date"] = pd.to_datetime(stock_data["date"])

In [51]:
df = stock_data.copy()
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

print(f"Date column type before: {stock_data['date'].dtype}")
print(f"Date column type after: {df['date'].dtype}")

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False,
)

print("Preprocessing data with FeatureEngineer...")
processed_data = fe.preprocess_data(df)

# Convert date to datetime after processing
processed_data["date"] = pd.to_datetime(processed_data["date"])
processed_data = processed_data.dropna().reset_index(drop=True)

print(f"Processed data shape: {processed_data.shape}")
print(f"Final date column type: {processed_data['date'].dtype}")
print(f"Features: {processed_data.columns.tolist()}")

processed_data.head()

Date column type before: datetime64[ns]
Date column type after: object
Preprocessing data with FeatureEngineer...
Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (6546, 8)
Successfully added vix
Successfully added turbulence index
Processed data shape: (45822, 18)
Final date column type: datetime64[ns]
Features: ['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma', 'vix', 'turbulence']


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,1998-12-22,11.746051,11.809159,11.706608,11.769716,15200,XLE,1,0.0,12.214153,11.522489,100.0,66.666667,100.0,11.746051,11.746051,22.780001,0.0
1,1998-12-22,14.582216,14.582216,14.533283,14.533283,600,XLI,1,0.0,12.214153,11.522489,100.0,66.666667,100.0,14.582216,14.582216,22.780001,0.0
2,1998-12-22,23.943213,24.281756,23.744756,24.211713,300500,XLK,1,0.0,12.214153,11.522489,100.0,66.666667,100.0,23.943213,23.943213,22.780001,0.0
3,1998-12-22,14.274879,14.291712,13.938207,13.938207,150300,XLP,1,0.0,12.214153,11.522489,100.0,66.666667,100.0,14.274879,14.274879,22.780001,0.0
4,1998-12-22,11.913816,12.082320,11.913816,12.082320,7900,XLU,1,0.0,12.214153,11.522489,100.0,66.666667,100.0,11.913816,11.913816,22.780001,0.0


In [52]:
processed_data = processed_data.sort_values(["date", "tic"], ignore_index=True)
processed_data.index = processed_data.date.factorize()[0]

cov_list = []
return_list = []

lookback = 252
for i in range(lookback, len(processed_data.index.unique())):
    data_lookback = processed_data.iloc[i - lookback : i]
    price_lookback = data_lookback.pivot(index="date", columns="tic", values="close")
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    cov = return_lookback.cov()
    cov_list.append(cov)

df_cov = pd.DataFrame(
    {
        "date": processed_data.date.unique()[lookback:],
        "cov_list": cov_list,
        "return_list": return_list,
    }
)
processed_data = processed_data.merge(df_cov, on="date")
processed_data = processed_data.sort_values(["date", "tic"]).reset_index(drop=True)

processed_data.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
0,1999-12-22,13.516106,13.628339,13.500072,13.548172,605300,XLE,2,-0.103611,14.166828,13.373424,45.574801,-104.984252,22.227769,13.977273,13.829793,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
1,1999-12-22,17.689077,17.788010,17.629718,17.738543,511700,XLI,2,0.010758,17.973613,17.176694,49.821595,11.633950,7.185969,17.667052,17.639551,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
2,1999-12-22,39.645401,40.112367,39.120064,39.972277,307700,XLK,2,1.361656,39.794935,34.553064,69.913462,160.096495,38.563849,36.267857,33.651680,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
3,1999-12-22,12.483123,12.576725,12.321446,12.363993,750600,XLP,2,-0.275550,14.056473,11.978243,42.296658,-110.385361,31.928400,13.197083,13.134717,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
4,1999-12-22,11.768043,11.889825,11.742404,11.857777,84600,XLU,2,-0.033414,12.055391,11.618068,47.905806,-59.159952,6.755132,11.878775,11.891842,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...


In [53]:
train = data_split(processed_data, TRAIN_START_DATE, end=TRAIN_END_DATE)
trade = data_split(processed_data, TRADE_START_DATE, end=TRADE_END_DATE)

In [ ]:
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv

In [ ]:
stock_dimensions = len(train.tic.unique())
state_space = stock_dimensions
print(f'Stock dimensions: {stock_dimensions}, State Space: {state_space}')

In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.005,
    "state_space": state_space,
    "stock_dim": stock_dimensions,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimensions,
    "reward_scaling": 1e-4,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

In [ ]:
models_to_train = {
    "PPO": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            "learning_rate": 0.0003,
            "n_steps": 2048,
            "batch_size": 64,
            "n_epochs": 10,
            "gamma": 0.99,
            "gae_lambda": 0.95,
            "clip_range": 0.2,
            "vf_coef": 0.5,
            "max_grad_norm": 0.5,
        },
    },
    "A2C": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            'learning_rate': 0.0007,
            'n_steps': 5,
            'gamma': 0.99,
            'gae_lambda': 1.0,
            'ent_coef':0.01,
            'vf_coef':0.25,
            'max_grad_norm':0.5,
        },
    },
    "DDPG": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            "learning_rate": 0.001,
            "buffer_size": 1000000,
            "learning_starts": 100,
            "batch_size": 100,
            "tau": 0.005,
            "gamma": 0.99,
        },
    },
}

In [ ]:
trained_models = {}
model_results = {}

for model_name, config in models_to_train.items():
    print(f"\n{'='*50}")
    print(f"Training {model_name} model...")
    print(f"{'='*50}")

    try:
        agent = DRLAgent(env=e_train_gym)

        model = agent.get_model(
            model_name=model_name.lower(),
            policy=config["policy"],
            model_kwargs=config["model_kwargs"],
        )

        trained_model = agent.train_model(
            model=model,
            total_timesteps=config["total_timesteps"],
            tb_log_name=model_name.lower(),
        )

        model_path = f"{TRAINED_MODEL_DIR}/{model_name.lower()}_ff_model"
        trained_model.save(model_path)
        trained_models[model_name] = trained_model

        print(f"{model_name} training completed and saved!")

    except Exception as e:
        print(f"Error training {model_name}: {str(e)}")
        continue

print(f"\nSuccessfully trained {len(trained_models)} models")

In [ ]:
def test_model(model, model_name, env):
    """ "
    Test a trained model return results
    """
    print(f"\nTesting {model_name}...")

    obs = env.reset()

    for i in range(len(env.get_attr("df")[0].index.unique()) - 1):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones:
            break

    asset_memory = env.get_attr("asset_memory")[0]
    daily_returns = np.diff(asset_memory) / asset_memory[:-1]

    results = {
        "final_value": asset_memory[-1],
        "total_return": (asset_memory[-1] / env.get_attr("initial_amount")[0] - 1)
        * 100,
        "daily_returns": daily_returns,
        "asset_memory": asset_memory,
        "date_memory": env.get_attr("date_memory")[0],
        "actions_memory": env.get_attr("actions_memory")[0],
    }
    return results

In [ ]:
test_results = {}

for model_name, model in trained_models.items():
    try:
        # Create a vectorized environment for testing
        test_env = DummyVecEnv([lambda: e_trade_gym])

        # Reset test environment
        obs = test_env.reset()

        # Test model
        results = test_model(model, model_name, test_env)
        test_results[model_name] = results

        print(f"{model_name} Results:")
        print(f" Final Portfolio Value: ${results['final_value']:,.2f}")
        print(f" Total Return: {results['total_return']:.2f}%")

        # Calculate additional metrics
        daily_returns = pd.Series(results["daily_returns"])
        sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
        max_drawdown = daily_returns.cumsum().min()

        print(f" Sharpe Ratio: {sharpe_ratio:.4f}")
        print(f" Max Drawdown: {max_drawdown:.4f}")

    except Exception as e:
        print(f"Error testing {model_name}: {str(e)}")
        continue

In [ ]:
# Create comparsion dataframe
comparison_data = []
for model_name, results in test_results.items():
    daily_returns = pd.Series(results['daily_returns'])
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
    max_drawdown = (daily_returns.cumsum() - daily_returns.cumsum().expanding().max()).min()

    comparison_data.append({
        'Model': model_name,
        'Final Value': results['final_value'],
        'Total Return (%)': results['total_return'],
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Total Return (%)', ascending=False)

print("\nModel Performance Ranking:")
print(comparison_df.to_string(index=False))

# Save results
comparison_df.to_csv(f"{RESULTS_DIR}/industry_model_comparison_{lookback}_window.csv", index=False)

# Create performance visualization
plt.figure(figsize=(15, 10))

# Plot 1: Portfolio Value Over Time
plt.subplot(2, 2, 1)
for model_name, results in test_results.items():
    dates = pd.to_datetime(results['date_memory'])
    values = results['asset_memory']
    plt.plot(dates, values, label=f"{model_name}", linewidth=2)

plt.title('Portfolio Value Over Time (Industry Model: {} Window)'.format(lookback))
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Plot 2: Total Returns Comparison
plt.subplot(2, 2, 2)
models = comparison_df['Model']
returns = comparison_df['Total Return (%)']
colors = plt.cm.viridis(np.linspace(0, 1, len(models)))
bars = plt.bar(models, returns, color=colors)
plt.title('Total Returns Comparison')
plt.xlabel('Model')
plt.ylabel('Total Return (%)')
plt.xticks(rotation=45)
for bar, ret in zip(bars, returns):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{ret:.1f}%', ha='center', va='bottom')

# Plot 3: Sharpe Ratio Comparison
plt.subplot(2, 2, 3)
sharpe_ratios = comparison_df['Sharpe Ratio']
bars = plt.bar(models, sharpe_ratios, color=colors)
plt.title('Sharpe Ratio Comparison')
plt.xlabel('Model')
plt.ylabel('Sharpe Ratio')
plt.xticks(rotation=45)
for bar, sharpe in zip(bars, sharpe_ratios):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{sharpe:.3f}', ha='center', va='bottom')

# Plot 4: Max Drawdown Comparison (negative values, so we flip for visualization)
plt.subplot(2, 2, 4)
drawdowns = comparison_df['Max Drawdown'].abs()
bars = plt.bar(models, drawdowns, color=colors)
plt.title('Max Drawdown Comparison')
plt.xlabel('Model')
plt.ylabel('Max Drawdown (abs)')
plt.xticks(rotation=45)
for bar, dd in zip(bars, drawdowns):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{dd:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig(f"{RESULTS_DIR}/industry_model_comparison_{lookback}_window.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import yfinance as yf

data = yf.download(TICKERS, start='2005-12-31', end='2024-12-31')['Close']

[*********************100%***********************]  10 of 10 completed


In [7]:
# Calculate returns from price data
# Get Close prices and calculate returns

df = data.copy()

df = df.pct_change() * 100

# Drop any NaN values that occur on the first day
df = df.dropna()
df.reset_index(inplace=True)
df.head()

Ticker,Date,IYZ,VTI,XLE,XLI,XLK,XLP,XLU,XLV,XLY,XRT
0,2006-06-23,0.201306,0.153558,2.221367,0.271473,-0.298632,-0.335000,0.158184,0.100190,-0.091151,0.457958
1,2006-06-26,-0.160714,0.490379,1.105299,0.270763,0.199687,0.546214,0.410576,-0.366924,0.517013,0.000000
2,2006-06-27,-0.764598,-0.951767,0.185264,-0.930101,-1.345271,-0.459679,-0.377442,-1.037844,-1.270751,-1.877147
3,2006-06-28,0.973220,0.708441,1.720008,0.302833,0.656553,0.461801,0.284160,0.000000,0.091934,-0.027332
4,2006-06-29,1.686763,2.183233,2.836355,2.113557,2.558914,1.587937,1.290914,1.725329,2.173876,2.542337


In [14]:
start_date = pd.to_datetime('2019-01-01')
end_date = pd.to_datetime('2024-12-21')

df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].reset_index(drop=True)

df['Date'] = pd.to_datetime(df['Date'])
df = df.reset_index(drop=True)
df.head()

Ticker,Date,IYZ,VTI,XLE,XLI,XLK,XLP,XLU,XLV,XLY,XRT
0,2019-01-02,0.759006,0.078340,1.970373,0.512330,0.064519,-0.590805,-1.719577,-1.514336,0.757492,1.414968
1,2019-01-03,-1.544264,-2.317396,-0.991784,-3.042942,-5.046768,-0.574457,-0.019216,-2.030494,-2.165202,-1.395226
2,2019-01-04,3.060442,3.310093,3.402395,3.791633,4.432027,2.131906,1.480762,2.983120,3.309443,3.073924
3,2019-01-07,1.410553,1.008523,1.486559,0.813499,0.894293,-0.136555,-0.682212,0.383870,2.261234,3.100582
4,2019-01-08,1.537331,1.029199,0.773546,1.370288,0.838022,0.918157,1.240221,0.776422,1.105617,0.872370


In [15]:
T, N = df.shape  # Recalculate total periods
N = N - 1  # Adjust for Date column
df.head()

Ticker,Date,IYZ,VTI,XLE,XLI,XLK,XLP,XLU,XLV,XLY,XRT
0,2019-01-02,0.759006,0.078340,1.970373,0.512330,0.064519,-0.590805,-1.719577,-1.514336,0.757492,1.414968
1,2019-01-03,-1.544264,-2.317396,-0.991784,-3.042942,-5.046768,-0.574457,-0.019216,-2.030494,-2.165202,-1.395226
2,2019-01-04,3.060442,3.310093,3.402395,3.791633,4.432027,2.131906,1.480762,2.983120,3.309443,3.073924
3,2019-01-07,1.410553,1.008523,1.486559,0.813499,0.894293,-0.136555,-0.682212,0.383870,2.261234,3.100582
4,2019-01-08,1.537331,1.029199,0.773546,1.370288,0.838022,0.918157,1.240221,0.776422,1.105617,0.872370


In [16]:
def getData(df, start, M):
    """returns excess returns for the N assets over a window of M periods from start"""
    return df.loc[start : (start + M - 1), df.columns != "Date"].astype(float).values


def naiveStrategy(N):
    """returns equal weights 1/N for N risky assets"""
    return np.ones(N) / N


def optimalWeights(m, c):
    """returns optimal normalized weights (equation 3 in DeMiguel)
    given mean vector m and covariance matrix c for N risky assets"""
    covI = np.linalg.inv(c) # inverse of covariance
    w = np.matmul(covI, m) # unnormalized optimal weights
    return w/sum(w) # normalized optimal weights


def meanVarianceStrategy(x):
    """returns optimal normalized weights for N risky assets 
    using Markowitz (1952) mean-variance strategy
    based on sample mean and covariance matrix of observations x"""
    m = np.mean(x,axis=0) # mean vector
    c = np.cov(x, rowvar=False)
    return optimalWeights(m, c)


def minVarianceStrategy(x):
    """Minimum variance portfolio (only uses covariance matrix)"""
    c = np.cov(x, rowvar=False)  # Covariance matrix
    ones = np.ones(len(c))       # Vector of ones
    covI = np.linalg.inv(c)
    w = np.matmul(covI, ones)    # Same as DeMiguel equation (8)
    return w/w.sum()             # Normalized weights


def returns(df, t, w):
    """Given weight vector w, computes returns in period t"""
    x = getData(df, t, 1)
    return np.matmul(x, w).item()

def SharpeRatio(returns):
    """returns Sharpe ratio given returns""" 
    m, s = np.mean(returns), np.std(returns)
    return m/s

def evaluateNaiveStrategy(M):
    w = naiveStrategy(N) # fixed weights
    res = []
    for t in range(T-M-2):
        x = getData(df, t, M)
        r = returns(df, t+M+1, w)
        res.append(r)
    SR = SharpeRatio(res)
    print('Sharpe Ratio for Naive Strategy = %4.4f' %SR)
    return SR

def evaluateMeanVarianceStrategy(M):
    resInSample, resOutOfSample = [], []
    for t in range(T-M-2):
        x = getData(df, t, M)
        w = meanVarianceStrategy(x)
        ri = returns(df, t+M, w)
        ro = returns(df, t+M+1, w)
        resInSample.append(ri)
        resOutOfSample.append(ro)
    SRI, SRO = SharpeRatio(resInSample), SharpeRatio(resOutOfSample)
    print(f'Sharpe Ratio for Mean Variance Strategy = {SRO:.4f}')
    return SRI, SRO


def evaluateMinVarianceStrategy(M):
    resOutOfSample = []
    for t in range(T-M-2):
        x = getData(df, t, M)
        w = minVarianceStrategy(x)
        ro = returns(df, t+M+1, w)
        resOutOfSample.append(ro)
    SR = SharpeRatio(resOutOfSample)
    print(f'Sharpe Ratio for Min Variance = {SR:.4f}')
    return SR

In [17]:
for M in range(60, 601, 60):
    print('\nWindow size = %d' % M)
    SR_naive = evaluateNaiveStrategy(M)
    SR_mv_insample, SR_mv = evaluateMeanVarianceStrategy(M)
    SR_minvar = evaluateMinVarianceStrategy(M)


Window size = 60
Sharpe Ratio for Naive Strategy = 0.0436
Sharpe Ratio for Mean Variance Strategy = 0.0144
Sharpe Ratio for Min Variance = 0.0249

Window size = 120
Sharpe Ratio for Naive Strategy = 0.0439
Sharpe Ratio for Mean Variance Strategy = 0.0159
Sharpe Ratio for Min Variance = 0.0134

Window size = 180
Sharpe Ratio for Naive Strategy = 0.0438
Sharpe Ratio for Mean Variance Strategy = -0.0395
Sharpe Ratio for Min Variance = 0.0093

Window size = 240
Sharpe Ratio for Naive Strategy = 0.0424
Sharpe Ratio for Mean Variance Strategy = -0.0268
Sharpe Ratio for Min Variance = 0.0075

Window size = 300
Sharpe Ratio for Naive Strategy = 0.0646
Sharpe Ratio for Mean Variance Strategy = 0.0295
Sharpe Ratio for Min Variance = 0.0186

Window size = 360
Sharpe Ratio for Naive Strategy = 0.0549
Sharpe Ratio for Mean Variance Strategy = 0.0488
Sharpe Ratio for Min Variance = 0.0069

Window size = 420
Sharpe Ratio for Naive Strategy = 0.0543
Sharpe Ratio for Mean Variance Strategy = -0.0495
S